# Waterbomb tessellation demo

In [1]:
%matplotlib widget
from bmcs_shell.api import WBCell5Param, WBTessellation5PV2, WBNumTessellation, WBNumTessellationInvest, WBTessellationBase, WBNumTessellationBase, WBCell5ParamV2, WBTessellation4P, WBCell4Param
from bmcs_utils.api import Extruder
import numpy as np
import k3d

## Analytical tessellation of symmetric waterbomb cell (4P cell)

In [2]:
def add_circle(plot, path, r):
    n = 100
#     path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
    first_contour = Extruder.get_circle_points(r = r, n=n)[int(n/2):,:]

    extruder = Extruder(first_contour, path)
    vertices, indices = extruder.get_triangulation_vertices_and_indices(with_ends=False)
    
    # extruder.show_in_k3d_as_surface(with_ends=False)
    
    mesh = k3d.mesh(vertices, indices,
                         color=0xc73737,
                         side='double')
    plot += mesh

In [3]:
# Cameras for k3d
# [
#  x1,y1,z1, # position of the camera in xyz space
#  x2,y2,z2, # the point where camera is currently looking at
#  x3,y3,z3  # orientation (up direction), this vector cannot be [0,0,0])
# ]
shell_side_view = [9153.397385449189, -7694.631708125074, 1127.588200496582,
                   0.000244140625, -0.00048828125, -2129.4488372802734,
                  -0.20092161158887856, 0.08487577463430307, 0.9759234646614198]
cell_front_view = [0, -2000, 2000,
                   0,     0,   0,
                   0,     0,   1]
shell_front_view = [13500, 0, -1000,
                    0,     0, -2500,
                    0,     0,   1]
shell_top_view = [0, 0, 13500,
                  0, 0, -2500,
                  -1,  0,   0]

In [6]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbt4p.pb.plot_fig, path = path, r=4200)

In [20]:
# wbt4p.pb.plot_fig.camera = shell_front_view
wbt4p.pb.plot_fig.camera = shell_top_view
# wbt4p.pb.plot_fig.camera = shell_side_view
# wbt4p.pb.plot_fig.camera = cell_front_view

In [4]:
wbt4p = WBTessellation4P(a = 500, b = 1000, c = 500, gamma = 1.0, n_phi_plus=7, n_x_plus = 5)
wbt4p.interact()
wbt4p.pb.plot_fig.screenshot_scale = 5.5
wbt4p.pb.plot_fig.grid_visible = False
wbt4p.pb.plot_fig.camera = shell_side_view

## Analytical tessellation of 5P cell V2

In [88]:
# wbttpv2.pb.plot_fig.camera = shell_front_view
wbttpv2.pb.plot_fig.camera = shell_top_view
# wbttpv2.pb.plot_fig.camera = shell_side_view
# wbttpv2.pb.plot_fig.camera = cell_front_view

In [19]:
path = np.array([[-4000, 0, -4000], [4000, 0, -4000]])
add_circle(wbttpv2.pb.plot_fig, path = path, r=4200)

In [20]:
wb_cell=WBCell5ParamV2(a = 500, b = 1000, c = 500, gamma = 1, beta= 1.878)
wbttpv2 = WBTessellation5PV2(wb_cell=wb_cell, n_x = 9, n_y = 7, sol_num=4)
wbttpv2.interact()
wbttpv2.pb.plot_fig.screenshot_scale = 5.5
wbttpv2.pb.plot_fig.grid_visible = False
wbttpv2.pb.plot_fig.camera = shell_side_view

W: 59619634888.31779
T_rho: 70580188.41484071
T_sigma: 71147282.36314777
sol_P2_t_2: 1.5640152437301034
sol_Q2_u_2: 1.584456103047374
Ana. solution: [-2.01560012 -2.00384626]


W: 64427414594.62382
T_rho: 81191609.88471349
T_sigma: 77224303.25291392
sol_P2_t_2: 1.4862886556109904
sol_Q2_u_2: 1.3842155881306837
Ana. solution: [-1.89034842 -1.95709601]
W: 65492383122.76767
T_rho: 85028224.14450406
T_sigma: 78904230.16848458
sol_P2_t_2: 1.453536860330539
sol_Q2_u_2: 1.3136939597033939
Ana. solution: [-1.84031576 -1.93637023]
W: 66033912659.9744
T_rho: 88040881.73002088
T_sigma: 80008520.91263677
sol_P2_t_2: 1.4262299576707258
sol_Q2_u_2: 1.2590603847479622
Ana. solution: [-1.79915114 -1.918599  ]
W: 65492383122.76767
T_rho: 85028224.14450406
T_sigma: 78904230.16848458
sol_P2_t_2: 1.453536860330539
sol_Q2_u_2: 1.3136939597033939
Ana. solution: [-1.84031576 -1.93637023]
W: 64427414594.62382
T_rho: 81191609.88471349
T_sigma: 77224303.25291392
sol_P2_t_2: 1.4862886556109904
sol_Q2_u_2: 1.3842155881306837
Ana. solution: [-1.89034842 -1.95709601]
W: 63276012732.60296
T_rho: 78064237.44655313
T_sigma: 75641880.05642931
sol_P2_t_2: 1.511230636311359
sol_Q2_u_2: 1.442473

Exception occurred in traits notification handler for event object: TraitChangeEvent(object=<bmcs_shell.folding.geometry.wb_cell.wb_cell_5p_v2.WBCell5ParamV2 object at 0x000001F2295642C0>, name='beta', old=1.65247768, new=1.62106178)
Traceback (most recent call last):
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\_trait_event_notifier.py", line 122, in __call__
    self.dispatcher(handler, event)
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\observe.py", line 27, in dispatch_same
    handler(event)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_tessellation_5pv2.py", line 30, in update_after_wb_cell_GEO_changes
    self.update_plot(self.pb)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_num_tessellation.py", line 120, in update_plot
    X_Ia, I_Fi  = self.calc_mesh_for_tessellated_cells()
  File "d:\hom

W: -19710017839.66886
T_rho: 24007797.448772352
T_sigma: 44470534.09095317
sol_P2_t_2: nan
sol_Q2_u_2: nan
Ana. solution: [nan nan]
W:

Exception occurred in traits notification handler for event object: TraitChangeEvent(object=<bmcs_shell.folding.geometry.wb_cell.wb_cell_5p_v2.WBCell5ParamV2 object at 0x000001F2295642C0>, name='beta', old=1.58964587, new=1.56451315)
Traceback (most recent call last):
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\_trait_event_notifier.py", line 122, in __call__
    self.dispatcher(handler, event)
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\observe.py", line 27, in dispatch_same
    handler(event)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_tessellation_5pv2.py", line 30, in update_after_wb_cell_GEO_changes
    self.update_plot(self.pb)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_num_tessellation.py", line 120, in update_plot
    X_Ia, I_Fi  = self.calc_mesh_for_tessellated_cells()
  File "d:\hom

...

W: -36471994496.83627
T_rho: 20264547.688017663
T_sigma: 42220097.98486983
sol_P2_t_2: nan
sol_Q2_u_2: nan
Ana. solution: [nan nan]


Exception occurred in traits notification handler for event object: TraitChangeEvent(object=<bmcs_shell.folding.geometry.wb_cell.wb_cell_5p_v2.WBCell5ParamV2 object at 0x000001F2295642C0>, name='beta', old=1.53938042, new=1.57079633)
Traceback (most recent call last):
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\_trait_event_notifier.py", line 122, in __call__
    self.dispatcher(handler, event)
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\observe.py", line 27, in dispatch_same
    handler(event)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_tessellation_5pv2.py", line 30, in update_after_wb_cell_GEO_changes
    self.update_plot(self.pb)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_num_tessellation.py", line 120, in update_plot
    X_Ia, I_Fi  = self.calc_mesh_for_tessellated_cells()
  File "d:\hom

W: -18864850741.044537
T_rho: 27147983.137700036
T_sigma: 52013716.14881604
sol_P2_t_2: nan
sol_Q2_u_2: nan
Ana. solution: [nan nan]
W: -5213282675.374687
T_rho: 33243557.94758382
T_sigma: 59692064.952707954
sol_P2_t_2: nan
sol_Q2_u_2: nan
Ana. solution: [nan nan]


Exception occurred in traits notification handler for event object: TraitChangeEvent(object=<bmcs_shell.folding.geometry.wb_cell.wb_cell_5p_v2.WBCell5ParamV2 object at 0x000001F2295642C0>, name='beta', old=1.57079633, new=1.59592905)
Traceback (most recent call last):
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\_trait_event_notifier.py", line 122, in __call__
    self.dispatcher(handler, event)
  File "C:\Users\spart\miniconda3\envs\bmcs_env\lib\site-packages\traits\observation\observe.py", line 27, in dispatch_same
    handler(event)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_tessellation_5pv2.py", line 30, in update_after_wb_cell_GEO_changes
    self.update_plot(self.pb)
  File "d:\homam\synced\programming\python\bmcs_shell\bmcs_shell\folding\geometry\wb_tessellation\wb_num_tessellation.py", line 120, in update_plot
    X_Ia, I_Fi  = self.calc_mesh_for_tessellated_cells()
  File "d:\hom

W: 11053644108.755531
T_rho: 41456964.43504916
T_sigma: 68784948.87351902
sol_P2_t_2: 0.5616915608866204
sol_Q2_u_2: 1.036092271457457
Ana. solution: [-1.60624511 -1.02355024]
W: 26143950171.611652
T_rho: 50131707.3512462
T_sigma: 76982821.44837892
sol_P2_t_2: 0.7318906483024463
sol_Q2_u_2: 1.209334553456145
Ana. solution: [-1.75973237 -1.26362005]
W: 37150760807.455
T_rho: 57231175.02100631
T_sigma: 82662668.06440642
sol_P2_t_2: 0.8400591554494894
sol_Q2_u_2: 1.2926701710325241
Ana. solution: [-1.82473235 -1.39738901]
W: 49285596167.642624
T_rho: 66047040.16417109
T_sigma: 88379702.01693167
sol_P2_t_2: 0.9584933075903131
sol_Q2_u_2: 1.3534602901823256
Ana. solution: [-1.86894294 -1.52841632]
W: 57457815800.96527
T_rho: 72827737.70162992
T_sigma: 91640923.63647534
sol_P2_t_2: 1.0441354740949556
sol_Q2_u_2: 1.3745580841197134
Ana. solution: [-1.88369426 -1.61397215]
W: 65396344054.07008
T_rho: 80608254.57504705
T_sigma: 93806739.96960482
sol_P2_t_2: 1.1418432540538073
sol_Q2_u_2: 1.3717

## General numerical tessellation of any wb cell with 6 facets

In [ ]:
wb_num_t = WBNumTessellation()
wb_num_t.interact()

## Class for investigating 3 cells tessellation manually

In [21]:
wb_num_t_invest = WBNumTessellationInvest()
wb_num_t_invest.interact()

---------------------------
smallest_dist= 0.003999999165102718
num_sol= [3.14159265 3.14159265]


---------------------------
smallest_dist= 0.0004678473701113329
num_sol= [3.0159872 3.0159872]
---------------------------
smallest_dist= 0.00016450978443090374
num_sol= [2.85954914 2.85954914]
---------------------------
smallest_dist= 0.0001477018390699791
num_sol= [2.735276 2.735276]
---------------------------
smallest_dist= 0.0001069054512607191
num_sol= [2.7043732 2.7043732]
---------------------------
smallest_dist= 9.912190456467206e-05
num_sol= [2.64279948 2.64279948]
---------------------------
smallest_dist= 0.00011139719956773245
num_sol= [2.61213763 2.61213763]
---------------------------
smallest_dist= 6.489959854368491e-05
num_sol= [2.58156497 2.58156495]
---------------------------
smallest_dist= 6.904283049526812e-05
num_sol= [2.55108527 2.55108527]
---------------------------
smallest_dist= 9.646735157322565e-05
num_sol= [2.52070257 2.52070257]
---------------------------
smallest_dist= 9.264448886086236e-05
num_sol= [2.4904206 2.4904206]
---------------------------


# Take gif animation from screenshots

In [82]:
import k3d
import numpy as np
from IPython.display import Image

points_number = 15
spread_range = 30
positions = spread_range * np.random.random_sample((points_number, 3)) - spread_range / 2
colors = np.random.randint(0, 0xFFFFFF, points_number)

plot = k3d.plot(height=240)
points = k3d.points(positions.astype(np.float32), colors.astype(np.uint32), point_size=3.0, shader='mesh')
plot += points

plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [66]:
import ipywidgets

plot.camera_auto_fit = False
plot.grid_auto_fit = False

speeds = -0.5 * np.sign(positions)
out = ipywidgets.Output()

@plot.yield_screenshots
def coroutine():
    global positions
    for i in range(12):
        positions += speeds
        points.positions = positions.astype(np.float32)
        plot.fetch_screenshot()
        screenshot = yield
        with open('screenshot_%03d.png'%i, 'wb') as f:
            f.write(screenshot)
        with out:
            print('screenshot_%03d.png saved.'%i)
    with out:
        print('done.')

coroutine()
print('loop is running in background...')
out

loop is running in background...


Output()